In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import shutil

In [40]:
underrepresented_classes = ['Lemongrass','Turmeric', 'Raddish', 'Seethaashoka', 'Pea', 
                            'Kasambruga', 'Malabar_Nut', 'Drumstick', 'Kambajala', 'Balloon_Vine']

In [41]:
source_directory = 'D:\MED_LEAF_ID\data\dataset\Medicinal Leaf dataset'

In [42]:
augmented_directory="D:\MED_LEAF_ID\data\\augmented"

In [43]:
os.makedirs(augmented_directory, exist_ok=True)


In [44]:
datagen = ImageDataGenerator(
    rotation_range=40,        # Random rotations
    width_shift_range=0.2,    # Random horizontal shifts
    height_shift_range=0.2,   # Random vertical shifts
    shear_range=0.2,          # Shear transformations
    zoom_range=0.2,           # Random zoom
    horizontal_flip=True,     # Random horizontal flip
    fill_mode='nearest'       # Fill any empty pixels created by transformations
)


In [45]:
for plant_name in underrepresented_classes:
    class_folder = os.path.join(source_directory, plant_name)
    augmented_class_folder = os.path.join(augmented_directory, plant_name)
    
    # Create the folder for augmented images if it doesn't exist
    os.makedirs(augmented_class_folder, exist_ok=True)
    
    # Loop through all images in the class folder and apply augmentation
    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        
        # Only process image files (skip non-image files)
        if img_name.endswith(('.jpg', '.png', '.jpeg')):
            img = load_img(img_path)  # Load the image
            img_array = img_to_array(img)  # Convert image to array
            img_array = img_array.reshape((1,) + img_array.shape)  # Reshape for the generator

            # Generate and save augmented images to the corresponding class folder
            i = 0
            for batch in datagen.flow(img_array, batch_size=1, save_to_dir=augmented_class_folder,
                                      save_prefix="aug_", save_format="png"):
                i += 1
                if i >= 10:  # Limit to 10 augmented images per original image
                    break